Code for successful analytics. Idea is to eventually rework into functions that can act on a single cumulative df of game shot

For this notebook, will be simply using a single games shot data

In [4]:
import pandas as pd
import networkx as nx

# Set display options to show all columns and a large width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Adjust the width as needed

shots = pd.read_csv('game_shots_20116.csv')
shots.head()


,Period,Time,Strength,Team,Shooter,Shot Type?,A1,A2,A3,A1 Zone,A2 Zone,A3 Zone,SC?,SOG?,Screen,Situation,Origin,Oddman,G?,Home Score State,Goalie,Game ID,Date,Home:,Away:
0,1,19:33,5v5,EDM,91,t,29.0,27.0,NaN,orrr,nl,NaN,NaN,NaN,NaN,r,nz,NaN,NaN,0,0,20116,10/27/2022,CHI,EDM
1,1,18:57,5v5,EDM,91,w,25.0,5.0,NaN,nl,dr,NaN,NaN,y,NaN,r,dz,NaN,NaN,0,0,20116,10/27/2022,CHI,EDM
2,1,18:40,5v5,EDM,37,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,y,NaN,f,oz,NaN,NaN,0,0,20116,10/27/2022,CHI,EDM
3,1,17:38,5v5,CHI,23,w,24.0,23.0,NaN,or,or,NaN,NaN,y,NaN,f,oz,NaN,NaN,0,0,20116,10/27/2022,CHI,EDM
4,1,17:24,5v5,CHI,4,w,43.0,NaN,NaN,opl,NaN,NaN,NaN,NaN,NaN,f,oz,NaN,NaN,0,0,20116,10/27/2022,CHI,EDM


In [5]:
shots['G?'].unique()

array([nan, 'y'], dtype=object)

# NetworkX functions

In [6]:
def create_hockey_network():
    """Create an empty directed graph for hockey analysis"""
    return nx.DiGraph()

def add_player_node(G, team, jersey_number):
    """Add a player node if it doesn't exist"""
    player_id = f"{team}_{jersey_number}"
    if not G.has_node(player_id):
        G.add_node(player_id, team=team, jersey_number=jersey_number, type='player')
    return player_id

def add_shot_node(G, shot_id, is_goal):
    """Add a shot node"""
    G.add_node(shot_id, type='shot', is_goal=is_goal)
    return shot_id

In [ ]:
def process_game_data(df):
    """Process game data and create a network graph"""
    # Create directed graph
    G = create_hockey_network()
    
    # Read CSV data
    #df = pd.read_csv(csv_data) if isinstance(csv_data, str) else pd.DataFrame(csv_data)
    
    # Track passing combinations for weight calculation
    passing_counts = {}
    
    # Process each shot attempt
    for idx, row in df.iterrows():
        # Create shot node
        shot_id = f"shot_{idx}"
        is_goal = True if row['G?'] == 'y' else False
        add_shot_node(G, shot_id, is_goal)
        
        # Add shooter node and connection to shot
        shooter_id = add_player_node(G, row['Team'], row['Shooter'])
        G.add_edge(shooter_id, shot_id, relationship_type='MAKES_SHOT')
        
        # Process passing sequence (A3 -> A2 -> A1 -> Shooter)
        passers = []
        for assist in ['A3', 'A2', 'A1']:
            if pd.notna(row[assist]):
                passer_id = add_player_node(G, row['Team'], int(row[assist]))
                passers.append(passer_id)
        
        # Add passing relationships
        if passers:
            # Include shooter in passing sequence
            passers.append(shooter_id)
            
            # Create edges for consecutive passes
            for i in range(len(passers)-1):
                passer = passers[i]
                receiver = passers[i+1]
                
                # Update passing counts
                pass_key = (passer, receiver)
                passing_counts[pass_key] = passing_counts.get(pass_key, 0) + 1
    
    # Add weighted passing relationships
    for (passer, receiver), weight in passing_counts.items():
        G.add_edge(passer, receiver, relationship_type='PASSES_TO', weight=weight)
    
    return G

In [8]:
def analyze_network(G):
    """Analyze the hockey network and return key metrics"""
    metrics = {
        'total_shots': len([n for n, attr in G.nodes(data=True) if attr['type'] == 'shot']),
        'total_goals': len([n for n, attr in G.nodes(data=True) if attr['type'] == 'shot' and attr['is_goal']]),
        'total_players': len([n for n, attr in G.nodes(data=True) if attr['type'] == 'player']),
        'passing_connections': len([e for e in G.edges(data=True) if e[2].get('relationship_type') == 'PASSES_TO']),
    }
    
    # Calculate player metrics
    player_metrics = {}
    for node in G.nodes():
        if G.nodes[node]['type'] == 'player':
            shots = len([e for e in G.edges(node) if G.edges[e]['relationship_type'] == 'MAKES_SHOT'])
            goals = len([e for e in G.edges(node) if 
                        G.edges[e]['relationship_type'] == 'MAKES_SHOT' and 
                        G.nodes[e[1]]['is_goal']])
            passes = sum(attr['weight'] for _, _, attr in G.edges(node, data=True) 
                        if attr.get('relationship_type') == 'PASSES_TO')
            
            player_metrics[node] = {
                'shots': shots,
                'goals': goals,
                'passes': passes
            }
    
    metrics['player_metrics'] = player_metrics
    return metrics

In [9]:
def visualize_passing_network(G):
    """Create a visualization of the passing network"""
    # Filter to just player nodes and passing relationships
    passing_network = G.copy()
    nodes_to_remove = [n for n, attr in passing_network.nodes(data=True) if attr['type'] == 'shot']
    passing_network.remove_nodes_from(nodes_to_remove)
    edges_to_remove = [e for e in passing_network.edges() if 
                      passing_network.edges[e].get('relationship_type') != 'PASSES_TO']
    passing_network.remove_edges_from(edges_to_remove)
    
    return passing_network

**page rank function**

In [17]:
def calculate_passing_influence(G, alpha=0.85):
    """
    Calculate passing influence scores using PageRank on the passing network
    
    Parameters:
    - G: Full game network
    - alpha: Damping factor for PageRank (default 0.85)
    
    Returns:
    - Dictionary of player IDs and their influence scores
    - Passing-only subgraph
    """
    # Create passing-only subgraph
    passing_network = G.copy()
    
    # Remove shot nodes and non-passing edges
    nodes_to_remove = [n for n, attr in passing_network.nodes(data=True) if attr['type'] == 'shot']
    passing_network.remove_nodes_from(nodes_to_remove)
    edges_to_remove = [e for e in passing_network.edges() if 
                      passing_network.edges[e].get('relationship_type') != 'PASSES_TO']
    passing_network.remove_edges_from(edges_to_remove)
    
    # Extract edge weights for PageRank
    edge_weights = {(u, v): d['weight'] for u, v, d in passing_network.edges(data=True)}
    
    # Calculate PageRank with edge weights
    influence_scores = nx.pagerank(passing_network, alpha=alpha, weight='weight')
    
    # Sort players by influence score
    ranked_players = sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)
    
    return ranked_players, passing_network

def analyze_passing_influence(G):
    """Analyze passing influence and return detailed metrics"""
    influence_rankings, passing_network = calculate_passing_influence(G)
    
    # Calculate additional passing metrics
    metrics = {
        'passing_influence': {},
        'network_stats': {
            'total_passes': sum(d['weight'] for _, _, d in passing_network.edges(data=True)),
            'unique_connections': passing_network.number_of_edges(),
            'active_passers': passing_network.number_of_nodes()
        }
    }
    
    # For each player, calculate comprehensive passing metrics
    for player_id, influence_score in influence_rankings:
        player_data = G.nodes[player_id]
        
        # Calculate direct passing stats
        passes_made = sum(d['weight'] for _, _, d in passing_network.edges(player_id, data=True))
        passes_received = sum(d['weight'] for _, _, d in passing_network.in_edges(player_id, data=True))
        
        metrics['passing_influence'][player_id] = {
            'influence_score': influence_score,
            'team': player_data['team'],
            'jersey_number': player_data['jersey_number'],
            'passes_made': passes_made,
            'passes_received': passes_received,
            'passing_ratio': passes_made / (passes_received + 1),  # Add 1 to avoid division by zero
        }
    
    return metrics

def format_influence_report(metrics):
    """Format the influence metrics into a readable report"""
    report = []
    report.append("=== Passing Influence Report ===\n")
    
    # Network-wide stats
    net_stats = metrics['network_stats']
    report.append(f"Total Passes: {net_stats['total_passes']}")
    report.append(f"Unique Passing Connections: {net_stats['unique_connections']}")
    report.append(f"Active Passers: {net_stats['active_passers']}\n")
    
    # Player rankings
    report.append("Player Rankings by Influence:")
    sorted_players = sorted(
        metrics['passing_influence'].items(),
        key=lambda x: x[1]['influence_score'],
        reverse=True
    )
    
    for player_id, stats in sorted_players:
        report.append(
            f"{stats['team']} #{stats['jersey_number']}: "
            f"Influence: {stats['influence_score']:.3f}, "
            f"Passes: {stats['passes_made']}, "
            f"Received: {stats['passes_received']}"
        )
    
    return "\n".join(report)

## using networkx functions

In [10]:
G = process_game_data(shots)
metrics = analyze_network(G)

In [11]:
passing_network = visualize_passing_network(G)

In [14]:
metrics

{'total_shots': 96,
 'total_goals': 11,
 'total_players': 36,
 'passing_connections': 98,
 'player_metrics': {'EDM_91': {'shots': 6, 'goals': 0, 'passes': 5},
  'EDM_27': {'shots': 2, 'goals': 0, 'passes': 1},
  'EDM_29': {'shots': 4, 'goals': 1, 'passes': 11},
  'EDM_5': {'shots': 1, 'goals': 0, 'passes': 3},
  'EDM_25': {'shots': 10, 'goals': 0, 'passes': 3},
  'EDM_37': {'shots': 2, 'goals': 0, 'passes': 0},
  'CHI_23': {'shots': 1, 'goals': 0, 'passes': 2},
  'CHI_24': {'shots': 2, 'goals': 0, 'passes': 3},
  'CHI_4': {'shots': 6, 'goals': 0, 'passes': 5},
  'CHI_43': {'shots': 2, 'goals': 0, 'passes': 5},
  'CHI_8': {'shots': 1, 'goals': 0, 'passes': 0},
  'CHI_58': {'shots': 1, 'goals': 0, 'passes': 1},
  'CHI_52': {'shots': 3, 'goals': 1, 'passes': 0},
  'EDM_97': {'shots': 4, 'goals': 3, 'passes': 16},
  'EDM_93': {'shots': 4, 'goals': 1, 'passes': 7},
  'CHI_88': {'shots': 7, 'goals': 1, 'passes': 12},
  'CHI_11': {'shots': 2, 'goals': 0, 'passes': 4},
  'CHI_19': {'shots': 3,

In [18]:
influence_metrics = analyze_passing_influence(G)

report = format_influence_report(influence_metrics)

In [20]:
print(report)

=== Passing Influence Report ===

Total Passes: 140
Unique Passing Connections: 98
Active Passers: 36

Player Rankings by Influence:
CHI #88: Influence: 0.116, Passes: 12, Received: 13
EDM #97: Influence: 0.095, Passes: 16, Received: 14
CHI #4: Influence: 0.065, Passes: 5, Received: 9
EDM #29: Influence: 0.060, Passes: 11, Received: 9
EDM #22: Influence: 0.055, Passes: 7, Received: 10
CHI #11: Influence: 0.053, Passes: 4, Received: 5
CHI #19: Influence: 0.052, Passes: 5, Received: 6
EDM #25: Influence: 0.046, Passes: 3, Received: 11
EDM #91: Influence: 0.042, Passes: 5, Received: 7
EDM #93: Influence: 0.032, Passes: 7, Received: 6
EDM #2: Influence: 0.031, Passes: 6, Received: 4
CHI #89: Influence: 0.029, Passes: 2, Received: 5
EDM #13: Influence: 0.024, Passes: 3, Received: 4
CHI #13: Influence: 0.024, Passes: 8, Received: 2
CHI #25: Influence: 0.022, Passes: 2, Received: 4
CHI #24: Influence: 0.021, Passes: 3, Received: 4
CHI #17: Influence: 0.021, Passes: 4, Received: 2
CHI #43: Inf

Top performing trios

In [24]:
import pandas as pd
import networkx as nx
from collections import defaultdict

def analyze_player_combinations_optimized(df, goal_weight=3):
    """
    Efficiently analyze player combinations directly from dataframe
    
    Parameters:
    - df: Pandas DataFrame of shot attempts
    - goal_weight: Weight multiplier for goals vs shots
    
    Returns:
    - Dictionary of pair and trio statistics
    """
    pair_stats = defaultdict(lambda: {'shots': 0, 'goals': 0, 'score': 0})
    trio_stats = defaultdict(lambda: {'shots': 0, 'goals': 0, 'score': 0})
    
    # Process each row directly
    for _, row in df.iterrows():
        # Get all players involved in the sequence
        players = []
        
        # Add shooter
        shooter_id = f"{row['Team']}_{row['Shooter']}"
        players.append(shooter_id)
        
        # Add assisters (if they exist)
        for assist in ['A1', 'A2', 'A3']:
            if pd.notna(row[assist]):
                player_id = f"{row['Team']}_{int(row[assist])}"
                players.append(player_id)
        
        # Score for this sequence
        is_goal = row['G?'] == 'y'
        sequence_score = goal_weight if is_goal else 1
        
        # Process pairs (any 2 players involved)
        if len(players) >= 2:
            for i in range(len(players)):
                for j in range(i + 1, len(players)):
                    pair = tuple(sorted([players[i], players[j]]))
                    pair_stats[pair]['shots'] += 1
                    if is_goal:
                        pair_stats[pair]['goals'] += 1
                    pair_stats[pair]['score'] += sequence_score
        
        # Process trios (any 3 players involved)
        if len(players) >= 3:
            for i in range(len(players)):
                for j in range(i + 1, len(players)):
                    for k in range(j + 1, len(players)):
                        trio = tuple(sorted([players[i], players[j], players[k]]))
                        trio_stats[trio]['shots'] += 1
                        if is_goal:
                            trio_stats[trio]['goals'] += 1
                        trio_stats[trio]['score'] += sequence_score
    
    return {
        'pairs': dict(pair_stats),
        'trios': dict(trio_stats)
    }

def get_top_combinations(combo_stats, n=5):
    """Get top n performing combinations efficiently"""
    # Process pairs
    top_pairs = sorted(
        [(pair, stats) for pair, stats in combo_stats['pairs'].items()],
        key=lambda x: (x[1]['score'], x[1]['goals']),
        reverse=True
    )[:n]
    
    # Process trios
    top_trios = sorted(
        [(trio, stats) for trio, stats in combo_stats['trios'].items()],
        key=lambda x: (x[1]['score'], x[1]['goals']),
        reverse=True
    )[:n]
    
    return {'pairs': top_pairs, 'trios': top_trios}

# Example usage
def analyze_game_combinations(df, goal_weight=3, top_n=5):
    """Quick analysis of top performing combinations"""
    # Read data
    #df = pd.read_csv(csv_data) if isinstance(csv_data, str) else pd.DataFrame(csv_data)
    
    # Get combination stats
    combo_stats = analyze_player_combinations_optimized(df, goal_weight)
    
    # Get top combinations
    return get_top_combinations(combo_stats, top_n)

In [26]:
top_combos = analyze_game_combinations(shots, goal_weight=3, top_n=5)

# Access results
for pair, stats in top_combos['pairs']:
    print(f"Pair {pair}: Goals: {stats['goals']}, Score: {stats['score']}")

for trio, stats in top_combos['trios']:
    print(f"Trio {trio}: Goals: {stats['goals']}, Score: {stats['score']}")

Pair ('CHI_13', 'CHI_88'): Goals: 3, Score: 13
Pair ('CHI_4', 'CHI_88'): Goals: 1, Score: 11
Pair ('CHI_19', 'CHI_88'): Goals: 1, Score: 9
Pair ('EDM_29', 'EDM_97'): Goals: 1, Score: 9
Pair ('EDM_22', 'EDM_29'): Goals: 1, Score: 9
Trio ('CHI_11', 'CHI_13', 'CHI_88'): Goals: 2, Score: 6
Trio ('CHI_19', 'CHI_4', 'CHI_88'): Goals: 1, Score: 6
Trio ('EDM_22', 'EDM_29', 'EDM_97'): Goals: 0, Score: 5
Trio ('CHI_13', 'CHI_19', 'CHI_88'): Goals: 1, Score: 4
Trio ('CHI_13', 'CHI_4', 'CHI_88'): Goals: 1, Score: 4


In [28]:
top_combos = analyze_game_combinations(shots, goal_weight=4, top_n=5)

# Access results
for pair, stats in top_combos['pairs']:
    print(f"Pair {pair}: Goals: {stats['goals']}, Score: {stats['score']}")

for trio, stats in top_combos['trios']:
    print(f"Trio {trio}: Goals: {stats['goals']}, Score: {stats['score']}")

Pair ('CHI_13', 'CHI_88'): Goals: 3, Score: 16
Pair ('CHI_4', 'CHI_88'): Goals: 1, Score: 12
Pair ('EDM_18', 'EDM_97'): Goals: 2, Score: 10
Pair ('EDM_2', 'EDM_97'): Goals: 2, Score: 10
Pair ('CHI_19', 'CHI_88'): Goals: 1, Score: 10
Trio ('CHI_11', 'CHI_13', 'CHI_88'): Goals: 2, Score: 8
Trio ('CHI_19', 'CHI_4', 'CHI_88'): Goals: 1, Score: 7
Trio ('CHI_13', 'CHI_19', 'CHI_88'): Goals: 1, Score: 5
Trio ('CHI_13', 'CHI_4', 'CHI_88'): Goals: 1, Score: 5
Trio ('EDM_22', 'EDM_29', 'EDM_97'): Goals: 0, Score: 5
